## Processing ATAC-seq Data into Base-Level Data
### Goal  
- The `goal` of this procedure is to transform raw single-nucleus ATAC-seq (snATAC-seq) data from `multiome` libraries into `base-level` chromatin accessibility data for downstream analysis. 
- The purpose is to work with finer resolution data (at the base level), rather than relying on peak calling, which groups accessible chromatin regions. 
- Base-level data offers a more `detailed view` of chromatin accessibility across individual bases, which can be critical for certain models like fine-mapping or multiomic studies, such as fitting the `mfsusie`.

### Data
- The `ROSMAP` single-nucleus ATAC-seq (`snATAC-seq`) is from the `10x Genomics` platform, specifically from the `multiome` libraries. 
- **Samples**: It containes` > 200 samples`.
- **Path**: `/mnt/mfs/ctcn/datasets/rosmap/snmultiome/dlpfc/batch1`    
    - fragment files: `/mnt/mfs/ctcn/datasets/rosmap/snmultiome/dlpfc/batch1/cellRanger/lirary_id/outs/atac_fragments.tsv.gz`   
    - sample data: `/mnt/mfs/ctcn/datasets/rosmap/snmultiome/dlpfc/batch1/demuxlet/scripts/SampleSheet.csv`   
    - library QCed Seurat object: `/mnt/mfs/ctcn/datasets/rosmap/snmultiome/dlpfc/batch1/atac_qc/analysis/step1_chromatin_assay_QC/snMultiome_chromatin_assay_libraryidtoid.rds`    
    - cell type annotated Seurat object: `/mnt/mfs/ctcn/datasets/rosmap/snmultiome/dlpfc/batch1/atac_qc/analysis/step3_postQC/'cell_type'/cell_type.rds`   

### Overview of the Process:
`Procedure`: fragnent QC(library) -> cell type annotation(library) ->  split by cell type and sample(6 cell type *240 samples) ->  Calculate coverage(6*240 bedgraph files) -> Merge all sample bedgraph into one for the sample cell type(6 bedgraph files)  ->  QC and get base by sample count matrix of each cell type 

`Input Data`: The starting data consists of `fragment` files generated from snATAC-seq. Each fragment file contains records of accessible chromatin fragments with the following information:    
- Chromosome
- Start and end positions of the fragment
- Barcode identifying the cell/nucleus
- Read count information

Data Structure:   
```bash
## Fragment file format:
chr start end barcode reads
chr1 10073 10210 AGTAGGATCCCGTTAC-1 1
chr1 10073 10309 TCGCGCACAGCAAATA-1 1
chr1 10079 10261 ACATCAATCCGCCAAA-1 2
chr1 10084 10204 TCATTGTTCCGCCTAT-1 1
chr1 10085 10303 ATTGGCTAGCGCCTAA-1 1
chr1 10085 10340 GAACTTATCCCTGATC-1 1
```
This format shows chromatin fragments for specific cells, identified by barcodes. The coordinates of the start and end positions represent where the chromatin is accessible.

`Purpose of Coverage`:
- Coverage is a measure of how many fragments (or reads) map to a specific base across all cells. It helps identify regions of high or low chromatin accessibility at the single-base resolution.  
- Tools like `bedtools`  are typically used to calculate coverage from fragment files.    
For example, calculating coverage will result in:   

```bash
| Base Position    | Sample1 |
|------------------|---------|
| chr1:100000      | 1       |
| chr1:100001      | 0       |
| chr1:100002      | 3       |
```
This output shows how many fragments overlap each base, giving a base-by-base accessibility profile.

### Procedure for Processing ATAC-seq Data to Base-Level Data

61 libraries(~ 4 samples/library) are processed in this procedure. The process involves several steps to transform raw fragment data into base-level chromatin accessibility data for each cell type. The steps are as follows:

**1) QC Fragment Files**:   

- Start with raw fragment files, which contain chromatin fragments with information such as chromosome positions, start and end points, and associated barcodes.   
- Apply quality control (QC) by matching filtered QCed barcodes (e.g., ncell, nfeature) from Seurat object metadata with the fragments. This ensures only high-quality cells are retained for further analysis.  

**2) Subset by Cell Type**:

- Use an annotation file (another Seurat object) to map barcodes to specific cell types.   
- Subset the fragments per cell type, ensuring each fragment is correctly assigned to its corresponding cell type based on the barcodes.  

**3) Subset by Sample**:   

- Further subset the cell-type-specific fragments by sample ID.   
- Each sample within the cell type is separated out to individual files for more granular analysis. 

**4) Run Coverage on Per-Sample Files**:   

- For each sample-specific fragment file, calculate base-level chromatin accessibility using bedtools coverage.
- `bedtools -bg`: for BedGraph output, which is more memory-efficient (1-2 mins/each).
- This produces per-sample `BedGraph` files, showing the chromatin accessibility at each base position.

**5) Merge BedGraph Files by Cell Type:**

- Merge the BedGraph files from different samples within the same cell type using `bedtools unionbedg`.
- This step combines data from multiple samples of the same cell type into a single base-by-sample matrix, ready for analysis.


```

**6) Quality Control:**

This pseudobulk-level matrix can undergo QC to filter out low-quality samples, ensuring reliable downstream analysis. The final matrix contains base-level chromatin accessibility for each sample within a given cell type.
- **Sample Quality Control (QC)**: Filtered out samples with fewer than 10 cells per cell type.
- **Phenotype Binning**: 
    - Binned phenotype data around each gene within ±25kb windows (total 50kb).
    - Over 20,000 genes were processed using this binning approach.
- **Batch Effect Adjustment**: 
    - Adjusted for batch effects using `ComBat_seq()` from `sva` package in R, specifically for count data.
    - Excluded batches with only a single sample, as most batches contain 3-5 samples; some variability remains post-filtering.
- **Hidden Factor Extraction** (using [Marchenko_PC method](https://github.com/cumc/xqtl-protocol/blob/main/code/data_preprocessing/covariate/covariate_hidden_factor.ipynb) ): 
    - Covariate Data: Processed covariates include `sex`, `age at death`, `PMI` (post-mortem interval), and `hidden factors` derived from phenotype data.
    - Missing data in covariates was filtered based on missing rate and subsequently imputed.
    - Covariate data sample sizes are aligned with the corresponding phenotype data sample sizes.



### Further background and introduction on QC background

### 1. Batch Effect Adjustment

#### Objective
The goal of batch effect adjustment is to remove systematic variation in count data caused by batch differences. These batch effects can arise from technical factors, such as variations in sample processing, which can confound the biological signals in the data. By adjusting for batch effects, we aim to ensure that observed differences in phenotype are more likely to reflect true biological variation rather than technical artifacts.

#### Principle
In high-throughput data, batch effects are often unavoidable due to practical limitations like processing constraints. **ComBat_seq** from the `sva` package in R is a method specifically designed to adjust for these batch effects in count data. It models the data to estimate and remove the batch effect, aiming to standardize data across batches. This approach uses empirical Bayes methods to improve estimates for smaller sample sizes, making it robust for typical biological datasets.

#### Method
**ComBat_seq** operates by:
- Modeling the data with batch and condition as covariates, estimating the batch effect parameters.
- Adjusting the data by removing batch-specific variability while preserving biological differences.

#### Workflow Steps
1. **Batch Definition**: Batches are defined based on the conditions under which samples were processed.
2. **Exclude Small Batches**: Batches with only a single sample are excluded from adjustment, as **ComBat_seq** requires multiple samples per batch for accurate estimation.
3. **Run ComBat_seq Adjustment**:
   - Apply **ComBat_seq** on the count data, specifying the batch.
   - Correct for systematic batch differences while preserving biological variability related to the conditions of interest.
4. **Check for Remaining Variability**:
   - After adjustment, examine the data for residual batch-related variability, as some minor differences may remain, especially in small sample sizes or less balanced batch designs.

#### Results
- **Batch-Adjusted Data**: A dataset in which batch-related effects are minimized, making the data more consistent across batches.
- **Consistency for Downstream Analysis**: By removing batch effects, the batch-adjusted data provides a more reliable basis for downstream analyses (e.g., differential expression or QTL analysis), enhancing the biological interpretability of observed patterns.


### 2. Hidden factor 
 
####  Background
In genomics and transcriptomics research, phenotype data may be influenced by batch effects, technical biases, and other unknown factors. These factors are known as "hidden factors" and introduce additional variability into the data, potentially obscuring true biological signals. To improve the accuracy of analysis, it is essential to remove the effects of known covariates on phenotype data and extract hidden factors.

#### Objective
The primary goals of this analysis are to:
- Remove the influence of known covariates from the phenotype data to obtain pure residuals.
- Identify and extract hidden factors in the data using Principal Component Analysis (PCA) to correct potential biases in subsequent analyses.

#### Inputs
- **Phenotype file (phenoFile)**: Contains phenotype data (count data in our analysis).
- **Covariate file (covFile)**: Contains known covariates such as sex, age at death, and PMI that may affect phenotype data.

#### Workflow Steps
The pipeline is available in [xqtl-protocal](https://github.com/cumc/xqtl-protocol/blob/main/code/data_preprocessing/covariate/covariate_hidden_factor.ipynb).
##### Step 1: Compute Residuals
- Match covariate and phenotype data to ensure sample consistency.
- Use a linear model to remove the influence of covariates from the phenotype data, yielding **residuals**.
- Residuals represent the part of phenotype data with known covariate effects removed, containing potential hidden factors and noise.

##### Step 2: Hidden Factor Analysis
- Perform PCA on the residuals from Step 1 to identify major sources of variability in the data.
- Use the Marchenko-Pastur distribution to select principal components that represent hidden factors.
- Retain these principal components as hidden factors and combine them with known covariates such as sex, age at death, and PMI, creating the final set of covariates for downstream analysis.



In [5]:
# Batch infomation: 61 batches in total, each batch contains 3-5 samples
library(data.table)
library(dplyr)
# Read batch information
sample_file <- "/mnt/mfs/ctcn/datasets/rosmap/snmultiome/dlpfc/batch1/experimentDesign/SampleSheet.csv"
sample <- fread(sample_file, colClasses = "character")
wgs_qc_file <- "/mnt/mfs/ctcn/datasets/rosmap/wgs/ampad/qualityControl/sampleSheetAfterQc.csv"
wgs_qc <- fread(wgs_qc_file, colClasses = "character")
sample <- sample %>%
  inner_join(wgs_qc) %>%
  select(SequencingID, SampleID)

# Extract batch information
batches <- sample$SequencingID
names(batches) <- sample$SampleID
print(batches)

Joining with `by = join_by(ProjID)`


   SM-CJGGO    SM-CJIZK    SM-CTDR8    SM-CJGLF    SM-CJK4X    SM-CJGNP 
    "RA004"     "RA004"     "RA004"     "RA004"     "RA006"     "RA006" 
   SM-CJIYI    SM-CTEET    SM-CJGNI    SM-CJIZH    SM-CJGM5    SM-CJIWV 
    "RA006"     "RA006"     "RA007"     "RA007"     "RA007"     "RA007" 
   SM-CJGNQ    SM-CJGI5    SM-CTDRG    SM-CJGHY    SM-CJGO3    SM-CJFO5 
    "RA008"     "RA008"     "RA008"     "RA008"     "RA009"     "RA009" 
MAP50106992    SM-CJIXG    SM-CJGLE    SM-CJEH2    SM-CTDRE    SM-CJIYG 
    "RA009"     "RA009"     "RA010"     "RA010"     "RA010"     "RA010" 
   SM-CJIY2    SM-CJIXW    SM-CTDRD    SM-CJIXQ ROS20376029    SM-CJEFR 
    "RA011"     "RA011"     "RA011"     "RA011"     "RA012"     "RA012" 
   SM-CJK4V    SM-CTDRA    SM-CJK5F    SM-CJK3J    SM-CJEJ6    SM-CJEKA 
    "RA012"     "RA012"     "RA012"     "RA013"     "RA013"     "RA013" 
   SM-CJIWS ROS11430815    SM-CTDSP    SM-CTEDQ    SM-CTDSL    SM-CJGGU 
    "RA013"     "RA014"     "RA014"     "RA014"    

In [8]:
## Final QCed binned data:
data <- readRDS("/home/al4225/project/multiome/step6_pseudoBulk/4_data_format/binned_57genes_region_data_6_celltype/combined_binned_57genes_region_data_6_celltype.rds")
head(names(data)) # 57 genes

first_element_name <- names(data)[1]
str(data[[first_element_name]])

[1] "chr1_207429229_207479229_ENSG00000117322"
[2] "chr1_207471146_207521146_ENSG00000203710"
[3] "chr10_11586753_11636753_ENSG00000148429" 
[4] "chr10_80429165_80479165_ENSG00000108219" 
[5] "chr10_96246586_96296586_ENSG00000095585" 
[6] "chr11_47353546_47403546_ENSG00000066336"

List of 13
 $ geno     : num [1:237, 1:888] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:237] "MAP26637867" "MAP50106992" "MAP61344957" "ROS11430815" ...
  .. ..$ : chr [1:888] "chr1:207429238_G_A" "chr1:207429252_G_A" "chr1:207429388_T_TAAAAA" "chr1:207429388_T_TAAAAAA" ...
 $ Ast_pheno: num [1:233, 1] 47018 6499 16584 18081 27869 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:233] "MAP26637867" "MAP50106992" "MAP61344957" "ROS11430815" ...
  .. ..$ : chr "chr1_207429229_207479229_ENSG00000117322"
 $ Ast_cov  : num [1:233, 1:7] 0 0 0 1 1 1 1 1 0 1 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:233] "SM-CJFK8" "SM-CTDUU" "SM-CJFLR" "SM-CJFM3" ...
  .. ..$ : NULL
 $ Exc_pheno: num [1:237, 1] 266395 29766 86178 84778 106969 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:237] "MAP26637867" "MAP50106992" "MAP61344957" "ROS11430815" ...
  .. ..$ : chr "chr1_207429229_207479229_ENSG00000117322"
 $ Exc_cov  : num [1:237, 1:6] 0

In [16]:
## Example of one data in one region
### Genotype Data
# For each gene, there is a single genotype file, which contains the union of samples across all phenotype data
# for the six cell types.
# col: variants within the region
# row: samples
geno_ENSG00000117322 = data$chr1_207429229_207479229_ENSG00000117322$geno
dim(geno_ENSG00000117322)
geno_ENSG00000117322[1:5,1:5]

[1] 237 888

,chr1:207429238_G_A,chr1:207429252_G_A,chr1:207429388_T_TAAAAA,chr1:207429388_T_TAAAAAA,chr1:207429391_AAAAG_A
MAP26637867,0,0,0,0,0
MAP50106992,0,0,0,0,0
MAP61344957,0,0,0,0,0
ROS11430815,0,1,0,0,0
ROS15738428,0,0,0,0,0


In [14]:
# phenotype data: Mic as an example
# col: region: chrom_WindowStart(gene pos -25kb)_WindowEnd(gene pos +25kb)_geneid, e.g. chr1_207429229_207479229_ENSG00000117322
# row: samples
Mic_pheno_ENSG00000117322 = data$chr1_207429229_207479229_ENSG00000117322$Mic_pheno 
dim(Mic_pheno_ENSG00000117322)
head(Mic_pheno_ENSG00000117322)


[1] 235   1

,chr1_207429229_207479229_ENSG00000117322
MAP26637867,89170
MAP50106992,7878
MAP61344957,38756
ROS11430815,22002
ROS15738428,31294
ROS20945666,35066


In [13]:
# covariates data: Mic as an example
# col: covariates(first 3 cols: msex, age_at_death, pmi; also with hidden factors from phenotype Mic)
# Note: it doen't contain colnames, but it will not use the col info, just used for regressing out cov from pheno.
# row: samples
Mic_cov_ENSG00000117322 = data$chr1_207429229_207479229_ENSG00000117322$Mic_cov
dim(Mic_cov_ENSG00000117322)
head(Mic_cov_ENSG00000117322)

[1] 235   6

SM-CJFK8,0,100.87337,7.500000,-11.17168,-1.2649857,3.5286763
SM-CTDUU,0,80.65708,5.500000,29.20719,-4.9717704,14.8556240
SM-CJFLR,0,94.31348,5.000000,102.58089,-3.0855194,0.8494127
SM-CJFM3,1,93.69199,7.250000,-187.80179,13.5051352,-18.5531397
SM-CTEET,1,87.92882,4.333333,-80.34725,13.8377383,-0.1811171
SM-CJGHP,1,90.52156,9.083333,218.66243,0.1136099,-4.0522568


### BedGraph Data Format and Explanation
- The `BedGraph` format is more memory-efficient. Unlike `base-level` data, which stores coverage at every base, BedGraph condenses `consecutive` positions with the `same` coverage value into a single `interval`, reducing redundancy. 
- This dramatically reduces memory usage, making it a better format for storing and processing large genomic datasets at scale. This format is particularly helpful during `QC`, and we can convert the interval data into base-level data for final analysis if needed.


#### Bedgraph for one sample of Mic
- tool: `bedtools genomecov -bg`
```bash
head /home/al4225/project/multiome/step5_bedgraph_coverage/Microglia/MAP26637867.bedgraph
chrom   start   end MAP26637867(coverage)
chr1    10156   10157   1
chr1    10157   10169   2
chr1    10169   10186   3
chr1    10186   10209   2
chr1    10209   10210   1
chr1    10267   10279   1
chr1    10309   10436   2
chr1    17339   17494   2
chr1    87248   87394   1
chr1    91121   91362   2
```

#### Bedgraph after merging all samples of Mic, and get PICALM region
- tool: `bedtools unionbedg`  
- After generating the BedGraph files for all samples, I used bedtools unionbedg to merge them. The resulting file contains chromosome, start, end, and the coverage values for multiple samples. Each row represents an interval where the coverage values are the same across samples. Also memory savings, but still maintaining detailed coverage data.

- PICALM Region window:
    - Original PICALM region expanded by 1MB on both sides:
    - Start: chr11 84,668,727 (original: 85,668,727)
    - End: chr11 86,780,924 (original: 85,780,924)
    
```bash
zcat /home/al4225/project/multiome/step5_bedgraph_coverage/PICALM/PICALM_Microglia.bedgraph.gz |
head | cut -f 1-6
chrom   start             end    MAP26637867         MAP50106992      MAP61344957
chr11   84668727        84668728        1               0               1
chr11   84668728        84668729        1               0               1
chr11   84668729        84668735        1               0               1
chr11   84668735        84668736        1               0               1
chr11   84668736        84668741        1               0               1
chr11   84668741        84668747        1               0               1
chr11   84668747        84668750        1               0               1
chr11   84668750        84668751        1               0               1
chr11   84668751        84668752        1               0               1
```

- `chrom`: Chromosome name (e.g., chr11 represents chromosome 11).
- `start`: Start position, 0-based. It indicates the first base position in the interval.
- `end`: End position, not inclusive (half-open interval).
- `Sample columns` (e.g., MAP26637867, MAP50106992, MAP61344957): These columns represent the coverage value for  each specific sample. The numbers indicate the signal strength or the number of fragments covering that region.   

For example, chr11 84668727 84668728 1 0 1 means that, for the region on chromosome 11 from 84668727 to 84668728, the samples MAP26637867 and MAP61344957 have a coverage of 1, while MAP50106992 has a coverage of 0.

### PICALM genotype data
- ROSMAP PLINK Genotype Data:
    - Subsetted data covering 1153 samples (rows) and 74,512 SNPs (columns) in the region.
    - Need to map samples to each phenotype data.
```bash
zcat /home/al4225/project/multiome/step5_bedgraph_coverage/PICALM_genotype/genotype/snuc_pseudo_bulk_mega.ENSG00000073921.PICALM_ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.11.tsv.gz | head -n 5| cut -f 1-4
chr11:84668727_C_T      chr11:84668730_T_C      chr11:84668839_C_T      chr11:84668840_G_A
MAP15387421     0       0       0
MAP26637867     0       0       0
MAP29629849     0       0       0
MAP33332646     0       0       0
```